# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())
assert('Apache II score' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [2]:
import json
import doctest
apache = 'apache.json'

In [3]:
### BEGIN SOLUTION
def score1(config_file, fields, result = -1):
    """ (int) -> json, str, float or int
    This function will look throught the jason file input are reports the score for the range that the rusult is within.  
    This will take the input for all of the Apache II inpurt that only have a single input and return the score.  The string
    input for field is not case sensative.
    
    >>> score1(apache, 'age', 30)
    0
    >>> score1(apache, 'age', 57) 
    3
    >>> score1(apache, 'age', 88) 
    6
    >>> score1(apache, 'teMperature', 39.1) 
    3
    >>> score1(apache, 'temperature', 34) 
    1
    >>> score1(apache, 'HEART RATE', 73)
    0
    >>> score1(apache, 'Heart rate', 51)
    3
    >>> score1(apache, 'rESPIRATORY rate', 70) 
    4
    >>> score1(apache, 'Respiratory Rate', 15) 
    0
    >>> score1(apache, 'ph', 9) 
    4
    >>> score1(apache, 'pH', 7.35) 
    0
    >>> score1(apache, 'sodium', 156) 
    2
    >>> score1(apache, 'sodium', 137) 
    0
    >>> score1(apache, 'Potassium', 5.8) 
    1
    >>> score1(apache, 'Potassium', 2)
    4
    >>> score1(apache, 'Hematocrit', 55)
    2
    >>> score1(apache, 'Hematocrit', 24)
    2
    >>> score1(apache, 'White blood count', 50)
    4
    >>> score1(apache, 'White blood count', 17)
    1
    """
    config = json.load(open(config_file))
    score = 0
    if fields == 'ph' or fields == 'PH' or fields == 'Ph' or fields == 'pH':
        field = config.get('pH')
    else:
        field = config.get(fields.title())      
    for i in field:
        if i.get('min') <= result and i.get('max') > result:
            score = i.get('points')
    return score


In [4]:
doctest.run_docstring_examples(score1, globals(), verbose=True)

Finding tests in NoName
Trying:
    score1(apache, 'age', 30)
Expecting:
    0
ok
Trying:
    score1(apache, 'age', 57) 
Expecting:
    3
ok
Trying:
    score1(apache, 'age', 88) 
Expecting:
    6
ok
Trying:
    score1(apache, 'teMperature', 39.1) 
Expecting:
    3
ok
Trying:
    score1(apache, 'temperature', 34) 
Expecting:
    1
ok
Trying:
    score1(apache, 'HEART RATE', 73)
Expecting:
    0
ok
Trying:
    score1(apache, 'Heart rate', 51)
Expecting:
    3
ok
Trying:
    score1(apache, 'rESPIRATORY rate', 70) 
Expecting:
    4
ok
Trying:
    score1(apache, 'Respiratory Rate', 15) 
Expecting:
    0
ok
Trying:
    score1(apache, 'ph', 9) 
Expecting:
    4
ok
Trying:
    score1(apache, 'pH', 7.35) 
Expecting:
    0
ok
Trying:
    score1(apache, 'sodium', 156) 
Expecting:
    2
ok
Trying:
    score1(apache, 'sodium', 137) 
Expecting:
    0
ok
Trying:
    score1(apache, 'Potassium', 5.8) 
Expecting:
    1
ok
Trying:
    score1(apache, 'Potassium', 2)
Expecting:
    4
ok
Trying:
    score1

In [5]:
def OFH(config_file, surg = 'None'):
    """ (int) -> json, str
    This function will look throught the jason file input are reports History of severe organ insufficiency or immunocompromised and if the there was a surgical procedure 
    or non-operative treatment.  The function reports the score associated with the presents of orgain insufficiency and operative treatment.  If there is no input for surgery,
    surgery will default to None.
    
    
    >>> OFH(apache, "nonoperative")
    5
    >>> OFH(apache, "emergency")
    5
    >>> OFH(apache, "elective")
    2
    >>> OFH(apache, )
    0
    """
    surg = surg.title()
    config = json.load(open(config_file))
    organ = config.get("Organ Failure History")

    score = organ.get(surg)
    if score == None:
        score = 0
    return score


In [6]:
doctest.run_docstring_examples(OFH, globals(), verbose=True)

Finding tests in NoName
Trying:
    OFH(apache, "nonoperative")
Expecting:
    5
ok
Trying:
    OFH(apache, "emergency")
Expecting:
    5
ok
Trying:
    OFH(apache, "elective")
Expecting:
    2
ok
Trying:
    OFH(apache, )
Expecting:
    0
ok


In [7]:

def oxy(config_file, fio2,  pao2 = 0, a_a = 0):
    """ (int) -> json, int, int, int
    This function will look throught the jason file input are reports the score associated with FiO2.  It separates Oxygenation based on FiO2.  If FiO2 <50%,  the function will use PaO2
    otherwise, it will use the A-a gradient.  It will return the Apache II score associates with these inputs.
    
    >>> oxy(apache, 60, 45, 550)
    4
    >>> oxy(apache, 70, 33, 368)
    3
    >>> oxy(apache, 55, 322, 222)
    2
    >>> oxy(apache, 51, 56, 124)
    0
    >>> oxy(apache, 49, 65, 6666)
    1
    >>> oxy(apache, 20, 57, 45)
    3
    >>> oxy(apache, 37, 40, 23)
    4
    >>> oxy(apache, 1, 65, 111)
    1
    """
    config = json.load(open(config_file))
    fio2_rules = config.get("FiO2")
    oxy_rul = {}
    score = 0
    n = 0
    for fio2_rul in fio2_rules:
        n = 1 + n
        if fio2 >= fio2_rul.get('min') and fio2 < fio2_rul.get('max') and n == 1:
            oxy_rul = fio2_rul.get('PaO2')
            x = pao2
        elif fio2 >= fio2_rul.get('min') and fio2 < fio2_rul.get('max') and n == 2:
            oxy_rul = fio2_rul.get('A-a gradient')
            x = a_a
    for oxy_p in oxy_rul:
        if x >= oxy_p.get('min') and x < oxy_p.get('max'):
            score = oxy_p.get('points')
    return score

In [8]:
doctest.run_docstring_examples(oxy, globals(), verbose=True)

Finding tests in NoName
Trying:
    oxy(apache, 60, 45, 550)
Expecting:
    4
ok
Trying:
    oxy(apache, 70, 33, 368)
Expecting:
    3
ok
Trying:
    oxy(apache, 55, 322, 222)
Expecting:
    2
ok
Trying:
    oxy(apache, 51, 56, 124)
Expecting:
    0
ok
Trying:
    oxy(apache, 49, 65, 6666)
Expecting:
    1
ok
Trying:
    oxy(apache, 20, 57, 45)
Expecting:
    3
ok
Trying:
    oxy(apache, 37, 40, 23)
Expecting:
    4
ok
Trying:
    oxy(apache, 1, 65, 111)
Expecting:
    1
ok


In [9]:
def creatin(config_file, creatinine = 0, failure = ''):
    """ (int) -> json, float, str
    This function will look throught the jason file input are reports the score associated with Serum creatinine.  The function will separate whether there is the presents 
    of renal failure (acute, chronic)
    >>> creatin(apache, 4.5, "ACUTE renal failure") 
    8
    >>> creatin(apache, 2.5, "ACUTE renal failure")
    6
    >>> creatin(apache, 1.7, "ACUTE renal failure")
    4
    >>> creatin(apache, 4, "CHRONIC renal failure")
    4
    >>> creatin(apache, 2.2, "CHRONIC renal failure")
    3
    >>> creatin(apache, 1.7, "CHRONIC renal failure")
    2
    >>> creatin(apache, 1.2)
    0
    >>> creatin(apache, 0.5)
    2
    """
    config = json.load(open(config_file))
    renal_failure = config.get("Creatinine")
    
    score = 0
    renal_rule = renal_failure.get(failure.title())
    for crit in renal_rule:
        if creatinine >= crit.get('min') and creatinine < crit.get('max'):
            score = crit.get('points')
    return score

In [10]:
doctest.run_docstring_examples(creatin, globals(), verbose=True)

Finding tests in NoName
Trying:
    creatin(apache, 4.5, "ACUTE renal failure") 
Expecting:
    8
ok
Trying:
    creatin(apache, 2.5, "ACUTE renal failure")
Expecting:
    6
ok
Trying:
    creatin(apache, 1.7, "ACUTE renal failure")
Expecting:
    4
ok
Trying:
    creatin(apache, 4, "CHRONIC renal failure")
Expecting:
    4
ok
Trying:
    creatin(apache, 2.2, "CHRONIC renal failure")
Expecting:
    3
ok
Trying:
    creatin(apache, 1.7, "CHRONIC renal failure")
Expecting:
    2
ok
Trying:
    creatin(apache, 1.2)
Expecting:
    0
ok
Trying:
    creatin(apache, 0.5)
Expecting:
    2
ok


In [11]:
def apache_outcome_data(config_file, a_score, ofh):
    """ (int) -> json, float, str
    This function takes the apach json file and returns the associated mortality outcomes.
    """
    of1 = ''
    config = json.load(open(config_file))
    apache_outcome = config.get("Apache II score")
    if ofh != apache_outcome.get("Nonoperative"):
        ofh1 = apache_outcome.get("Postoperative")
    else:
        ofh1 = apache_outcome.get("Nonoperative")
    for surg in ofh1:
        if a_score >= surg.get('min') and a_score < surg.get('max'):
            outcome = surg.get('rate')
    return outcome


In [12]:
assert(apache_outcome_data(apache, 45, "Nonoperative") == '88%')

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [13]:
assert(score1(apache, 'age', 30) == 0)
assert(score1(apache, 'age', 48) == 2)
assert(score1(apache, 'age', 57) == 3)
assert(score1(apache, 'age', 66) == 5)
assert(score1(apache, 'age', 88) == 6)

assert(score1(apache, 'temperature', 41.5) == 4)
assert(score1(apache, 'teMperature', 39.1) == 3)
assert(score1(apache, 'Temperature', 38.7) == 1)
assert(score1(apache, 'temperature', 37) == 0)
assert(score1(apache, 'temperature', 34) == 1)
assert(score1(apache, 'temperature', 33) == 2)
assert(score1(apache, 'temperature', 31) == 3)
assert(score1(apache, 'temperature', 20) == 4)

assert(score1(apache, 'Heart rate', 200) == 4)
assert(score1(apache, 'Heart Rate', 155) == 3)
assert(score1(apache, 'Heart rate', 111) == 2)
assert(score1(apache, 'HEART RATE', 73) == 0)
assert(score1(apache, 'Heart rate', 60) == 2)
assert(score1(apache, 'Heart rate', 51) == 3)
assert(score1(apache, 'Heart rate', 2) == 4)

assert(score1(apache, 'rESPIRATORY rate', 70) == 4)
assert(score1(apache, 'Respiratory rate', 44) == 3)
assert(score1(apache, 'Respiratory Rate', 27) == 1)
assert(score1(apache, 'Respiratory Rate', 15) == 0)
assert(score1(apache, 'Respiratory Rate', 11) == 1)
assert(score1(apache, 'Respiratory Rate', 7) == 2)
assert(score1(apache, 'Respiratory Rate', 5) == 4)

assert(score1(apache, 'ph', 9) == 4)
assert(score1(apache, 'pH', 7.69) == 3)
assert(score1(apache, 'Ph', 7.52) == 1)
assert(score1(apache, 'pH', 7.35) == 0)
assert(score1(apache, 'pH', 7.32) == 2)
assert(score1(apache, 'pH', 7.2) == 3)
assert(score1(apache, 'pH', 7) == 4)

assert(score1(apache, 'sodium', 200) == 4)
assert(score1(apache, 'sodium', 176) == 3)
assert(score1(apache, 'sodium', 156) == 2)
assert(score1(apache, 'sodium', 154) == 1)
assert(score1(apache, 'sodium', 137) == 0)
assert(score1(apache, 'sodium', 125) == 2)
assert(score1(apache, 'sodium', 115) == 3)
assert(score1(apache, 'sodium', 100) == 4)

assert(score1(apache, 'Potassium', 8) == 4)
assert(score1(apache, 'Potassium', 6.1) == 3)
assert(score1(apache, 'Potassium', 5.8) == 1)
assert(score1(apache, 'Potassium', 4) == 0)
assert(score1(apache, 'Potassium', 3.3) == 1)
assert(score1(apache, 'Potassium', 2.7) == 2)
assert(score1(apache, 'Potassium', 2) == 4)

assert(score1(apache, 'Hematocrit',66 ) == 4)
assert(score1(apache, 'Hematocrit', 55) == 2)
assert(score1(apache, 'Hematocrit', 48) == 1)
assert(score1(apache, 'Hematocrit', 35) == 0)
assert(score1(apache, 'Hematocrit', 24) == 2)
assert(score1(apache, 'Hematocrit', 10) == 4)

assert(score1(apache, 'White blood count', 50) == 4)
assert(score1(apache, 'White blood count', 23) == 2)
assert(score1(apache, 'White blood count', 17) == 1)
assert(score1(apache, 'White blood count', 11) == 0)
assert(score1(apache, 'White blood count', 2) == 2)
assert(score1(apache, 'White blood count', 0) == 4)

assert(OFH(apache, "nonoperative") == 5)
assert(OFH(apache, "emergency") == 5)
assert(OFH(apache, "elective") == 2)
assert(OFH(apache, ) == 0)

assert(oxy(apache, 60, 45, 550) == 4)
assert(oxy(apache, 70, 33, 368) == 3)
assert(oxy(apache, 55, 322, 222) == 2)
assert(oxy(apache, 51, 56, 124) == 0)
assert(oxy(apache, 49, 65, 6666) == 1)
assert(oxy(apache, 20, 57, 45) == 3)
assert(oxy(apache, 37, 40, 23) == 4)
assert(oxy(apache, 1, 65, 111) == 1)

assert(creatin(apache, 4.5, "ACUTE renal failure") == 8)
assert(creatin(apache, 2.5, "ACUTE renal failure") == 6)
assert(creatin(apache, 1.7, "ACUTE renal failure") == 4)
assert(creatin(apache, 4, "CHRONIC renal failure") == 4)
assert(creatin(apache, 2.2, "CHRONIC renal failure") == 3)
assert(creatin(apache, 1.7, "CHRONIC renal failure") == 2)
assert(creatin(apache, 1.2) == 0)
assert(creatin(apache, 0.5) == 2)



---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [14]:
def apache_score(config_file, ofh, age, temp, ph, h_r, r_r, na, k, creat, arf, hct, wbc, gcs, fi02, pa02, a_a_grad):
    """ (int) -> str, int, int, float, int, int, int, float, float, str, int, int, int, int, int, int
    This function will take the information:
        ofh = Organ Failure History (str)
        age = Age (int)
        temp = Temperature (int)
        ph = pH (float)
        h_r = Heart rate (int)
        r_r = Respiratory rate (int)
        na = Sodium (int)
        k = Potassium (float)
        creat = Creatinine (float)
        arf = Acute renal failure (str)
        hct = Hematocrit (int)
        wbc = White Blood Count (int)
        gcs = Glasgow Coma Scale (int)
        fi02 = FiO2 (int)
        pa02 = PaO2 (in)
        a_a_grad = A-a gradient (int)
    and returns the Apache II score for those patients.
    >>> apache_score("apache.json", "emergency", 6, 27, 7.3, 174, 10, 170, 7.8, 1.6, 'chronic renal failure', 3, 28, 13, 61, 59, 458)
    35
    >>> apache_score("apache.json", 'elective', 3, 4, 7.2, 74, 53, 106, 3.2, 1.2,  'acute renal failure', 60, 22, 11, 66, 59, 482) 
    31
    >>> apache_score("apache.json", 'nonoperative', 62, 28, 7.1, 56, 7, 177, 0.4, 1.6, 'acute renal failure', 38, 50, 7, 57, 74, 527)
    47
    >>> apache_score("apache.json", "emergency", 14, 9, 7.5, 63, 2, 51, 6.8, 1.2, 'acute renal failure', 12, 19, 2, 36, 56, 438)
    44
    """
    score = 0
    score += OFH(config_file, ofh)
    score += score1(config_file, 'Age', age)
    score += score1(config_file, 'Temperature', temp)
    score += score1(config_file, 'pH', ph)
    score += score1(config_file, 'Heart Rate', h_r)
    score += score1(config_file, 'Respiratory Rate', r_r)
    score += score1(config_file, 'Sodium', na)
    score += score1(config_file, 'Potassium', k)
    score += creatin(config_file, creat, arf)
    score += score1(config_file, 'Hematocrit', hct)
    score += score1(config_file, 'White Blood Count', wbc)
    score += 15 - gcs 
    score += oxy(config_file, fi02,  pa02, a_a_grad)

    return score

In [15]:
doctest.run_docstring_examples(apache_score, globals(), verbose=True)

Finding tests in NoName
Trying:
    apache_score("apache.json", "emergency", 6, 27, 7.3, 174, 10, 170, 7.8, 1.6, 'chronic renal failure', 3, 28, 13, 61, 59, 458)
Expecting:
    35
ok
Trying:
    apache_score("apache.json", 'elective', 3, 4, 7.2, 74, 53, 106, 3.2, 1.2,  'acute renal failure', 60, 22, 11, 66, 59, 482) 
Expecting:
    31
ok
Trying:
    apache_score("apache.json", 'nonoperative', 62, 28, 7.1, 56, 7, 177, 0.4, 1.6, 'acute renal failure', 38, 50, 7, 57, 74, 527)
Expecting:
    47
ok
Trying:
    apache_score("apache.json", "emergency", 14, 9, 7.5, 63, 2, 51, 6.8, 1.2, 'acute renal failure', 12, 19, 2, 36, 56, 438)
Expecting:
    44
ok


In [16]:
def apache_score_1(config_file, ofh, age, temp, ph, h_r, r_r, na, k, creat, arf, hct, wbc, gcs, fi02, pa02, a_a_grad):
    """ (int, str, str) -> str, int, int, float, int, int, int, float, float, str, int, int, int, int, int, int
    This function will take the information:
        ofh = Organ Failure History (str)
        age = Age (int)
        temp = Temperature (int)
        ph = pH (float)
        h_r = Heart rate (int)
        r_r = Respiratory rate (int)
        na = Sodium (int)
        k = Potassium (float)
        creat = Creatinine (float)
        arf = Acute renal failure (str)
        hct = Hematocrit (int)
        wbc = White Blood Count (int)
        gcs = Glasgow Coma Scale (int)
        fi02 = FiO2 (int)
        pa02 = PaO2 (in)
        a_a_grad = A-a gradient (int)
    and returns the Apache II score for those patients and if the information input is questionable.  This function will give the Apache II score
    as well as evaluate the input.  If the input would not be associate with survivable data, the function will report that this questionable input.
    The apache_score_1 created a list of tupples.  The a position in the tupple is the appache score, an int, and the b position in the tupple whether 
    the input is good or questionable, a str. The c position is the associated mortality the is associated with the operative or non-operative treatment and
    the score.
    """
    score = 0
    input = 'good'
    score += OFH(config_file, ofh)
    score += score1(config_file, 'Age', age)
    score += score1(config_file, 'Temperature', temp)
    score += score1(config_file, 'pH', ph)
    score += score1(config_file, 'Heart Rate', h_r)
    score += score1(config_file, 'Respiratory Rate', r_r)
    score += score1(config_file, 'Sodium', na)
    score += score1(config_file, 'Potassium', k)
    score += creatin(config_file, creat, arf)
    score += score1(config_file, 'Hematocrit', hct)
    score += score1(config_file, 'White Blood Count', wbc)
    score += 15 - gcs 
    score += oxy(config_file, fi02,  pa02, a_a_grad)
    score_o = score
    if age < 0 or age > 130 or temp < 25 or temp > 42 or ph < 6 or ph > 8.5 or na < 100 or na> 210 or h_r < 10 or h_r > 250 or r_r > 100 or r_r < 2 or hct < 10:
        input = 'questionable'
    outcome = apache_outcome_data(config_file, score_o, ofh)
    return score, input, outcome



In [17]:
apache_score_1(apache, "emergency", 6, 27, 7.3, 174, 10, 170, 7.8, 1.6, 'chronic renal failure', 3, 28, 13, 61, 59, 458)


(35, 'questionable', '88%')

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [18]:
assert (apache_score("apache.json", "emergency", 6, 27, 7.3, 174, 10, 170, 7.8, 1.6, 'chronic renal failure', 3, 28, 13, 61, 59, 458) == 35)
assert (apache_score("apache.json", 'elective', 3, 4, 7.2, 74, 53, 106, 3.2, 1.2,  'acute renal failure', 60, 22, 11, 66, 59, 482) == 31)
assert (apache_score("apache.json", 'nonoperative', 62, 28, 7.1, 56, 7, 177, 0.4, 1.6, 'acute renal failure', 38, 50, 7, 57, 74, 527) == 47)
assert (apache_score("apache.json", "emergency", 14, 9, 7.5, 63, 2, 51, 6.8, 1.2, 'acute renal failure', 12, 19, 2, 36, 56, 438) == 44)

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [19]:
def apache_cvs(http):
    """ (int) -> html
    This function will take the website for a cvs data which has information for patient with Apache II data
    and run it throught the apache_score function to output a list of appache data for those patients in oder
    of input.
    """
    import requests
    from contextlib import closing
    import csv
    url = http
    result = []
    with closing(requests.get(url, stream=True)) as r:
        f = (line.decode('utf-8') for line in r.iter_lines())
        reader = csv.reader(f, delimiter=',', quotechar='"')
        row = 1
        next(reader, None)
        for row in reader: 
            ofh = row[1]
            age = int(row[2])
            temp = int(row[3])
            ph = float(row[4])
            h_r = int(row[5])
            r_r = int(row[6])
            na = int(row[7])
            k = float(row[8])
            creat = float(row[9])
            arf = row[10]
            hct = int(row[11])
            wbc = float(row[12])
            gcs = int(row[13])
            fi02 = int(row[14])
            pa02 = int(row[15])
            a_a_grad = int(row[16])
            score = apache_score(apache, ofh, age, temp, ph, h_r, r_r, na, k, creat, arf, hct, wbc, gcs, fi02, pa02, a_a_grad)
            result.append(score)
    return result

test = apache_cvs("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")
print(test)

[35, 31, 47, 34, 44, 35, 31, 49, 40, 48, 42, 43, 32, 41, 42, 49, 37, 37, 38, 43, 41, 31, 38, 30, 41, 41, 34, 46, 40, 47, 36, 43, 41, 46, 44, 40, 39, 37, 41, 30, 46, 30, 41, 44, 35, 36, 40, 40, 30, 50, 52, 43, 46, 34, 33, 42, 41, 31, 46, 46, 34, 36, 33, 38, 26, 29, 46, 25, 40, 38, 39, 34, 39, 53, 44, 49, 37, 36, 36, 51, 33, 36, 43, 41, 24, 50, 29, 40, 36, 50, 29, 37, 34, 45, 34, 40, 37, 37, 47, 31, 33, 50, 28, 37, 33, 44, 40, 38, 40, 31, 44, 37, 44, 47, 25, 34, 32, 41, 38, 34, 38, 39, 41, 34, 29, 44, 22, 40, 34, 43, 40, 31, 35, 33, 27, 44, 52, 40, 53, 40, 40, 48, 31, 42, 49, 43, 35, 31, 36, 42, 34, 31, 30, 28, 38, 47, 32, 47, 31, 41, 40, 33, 35, 42, 49, 55, 51, 37, 53, 29, 33, 46, 36, 32, 33, 33, 42, 32, 41, 35, 39, 40, 40, 35, 28, 33, 42, 29, 30, 39, 34, 46, 30, 37, 31, 37, 41, 47, 45, 29, 45, 33, 33, 38, 28, 42, 32, 45, 36, 39, 43, 32, 37, 43, 34, 40, 27, 24, 39, 39, 35, 36, 39, 47, 27, 28, 50, 53, 36, 44, 38, 51, 43, 38, 38, 36, 40, 36, 40, 31, 39, 37, 38, 46, 28, 32, 37, 29, 26, 25,

In [20]:
def apache_cvs_1(http):
    """ (int, str, str) -> html
    This function will take the website for a cvs data which has information for patient with Apache II data
    and run it throught the apache_score function to output a list of apache data for those patients in order
    of input. 
    The result is a list of tupples.  The a position in the tupple is the apache score, an int, and the b position in the tupple whether 
    the input is good or questionable, a str. The c position is the associated mortality the is associated with the operative or non-operative treatment and
    the score.
    """
    import requests
    from contextlib import closing
    import csv
    url = http
    result = []
    out = []
    with closing(requests.get(url, stream=True)) as r:
        f = (line.decode('utf-8') for line in r.iter_lines())
        reader = csv.reader(f, delimiter=',', quotechar='"')
        row = 1
        next(reader, None)
        for row in reader: 
            ofh = row[1]
            age = int(row[2])
            temp = int(row[3])
            ph = float(row[4])
            h_r = int(row[5])
            r_r = int(row[6])
            na = int(row[7])
            k = float(row[8])
            creat = float(row[9])
            arf = row[10]
            hct = int(row[11])
            wbc = float(row[12])
            gcs = int(row[13])
            fi02 = int(row[14])
            pa02 = int(row[15])
            a_a_grad = int(row[16])
            score = apache_score_1(apache, ofh, age, temp, ph, h_r, r_r, na, k, creat, arf, hct, wbc, gcs, fi02, pa02, a_a_grad)
            result.append(score)
        return result

information_quality = apache_cvs_1("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")
for a, b, c in information_quality: ##this is a for loop that runs through the data and reports if the data input is good or questionable. Then it will report the score with associated mortality risk.
    print("Your Apache II result is {0}. Your input is {1}.".format(a, b))
    print("For the Apache II score {0}, the approximated in-hospital mortality rates are {1}.".format(a, c))


Your Apache II result is 35. Your input is questionable.
For the Apache II score 35, the approximated in-hospital mortality rates are 88%.
Your Apache II result is 31. Your input is questionable.
For the Apache II score 31, the approximated in-hospital mortality rates are 73%.
Your Apache II result is 47. Your input is good.
For the Apache II score 47, the approximated in-hospital mortality rates are 88%.
Your Apache II result is 34. Your input is good.
For the Apache II score 34, the approximated in-hospital mortality rates are 73%.
Your Apache II result is 44. Your input is questionable.
For the Apache II score 44, the approximated in-hospital mortality rates are 88%.
Your Apache II result is 35. Your input is questionable.
For the Apache II score 35, the approximated in-hospital mortality rates are 88%.
Your Apache II result is 31. Your input is questionable.
For the Apache II score 31, the approximated in-hospital mortality rates are 73%.
Your Apache II result is 49. Your input is 

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [21]:
def apache_scores(https):
    """
    (int) -> html
    This function will take the website for a cvs data which has information for test patient comparrison scores for the Apache II results
    
    """
    import requests
    from contextlib import closing
    import csv
    url = https
    result = []
    with closing(requests.get(url, stream=True)) as r:
        f = (line.decode('utf-8') for line in r.iter_lines())
        reader = csv.reader(f, delimiter=',', quotechar='"')
        next(reader, None)
        for row in reader: 
            test_score = float(row[0]) #needed to change from a str to and int
            test_score = int(test_score)
            result.append(test_score)
   #         print(test_score)
    return result
test = apache_scores("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv") #testin the function
print(test)

[35, 31, 47, 34, 44, 35, 31, 49, 40, 48, 42, 43, 32, 41, 42, 49, 37, 37, 38, 43, 41, 31, 38, 30, 41, 41, 34, 46, 40, 47, 36, 43, 41, 46, 44, 40, 39, 37, 41, 30, 46, 30, 41, 44, 35, 36, 40, 40, 30, 50, 52, 43, 46, 34, 33, 42, 41, 31, 46, 46, 34, 36, 33, 38, 26, 29, 46, 25, 40, 38, 39, 34, 39, 53, 44, 49, 37, 36, 36, 51, 33, 36, 43, 41, 24, 50, 29, 40, 36, 50, 29, 37, 34, 45, 34, 40, 37, 37, 47, 31, 33, 50, 28, 37, 33, 44, 40, 38, 40, 31, 44, 37, 44, 47, 25, 34, 32, 41, 38, 34, 38, 39, 41, 34, 29, 44, 22, 40, 34, 43, 40, 31, 35, 33, 27, 44, 52, 40, 53, 40, 40, 48, 31, 42, 49, 43, 35, 31, 36, 42, 34, 31, 30, 28, 38, 47, 32, 47, 31, 41, 40, 33, 35, 42, 49, 55, 51, 37, 53, 29, 33, 46, 36, 32, 33, 33, 42, 32, 41, 35, 39, 40, 40, 35, 28, 33, 42, 29, 30, 39, 34, 46, 30, 37, 31, 37, 41, 47, 45, 29, 45, 33, 33, 38, 28, 42, 32, 45, 36, 39, 43, 32, 37, 43, 34, 40, 27, 24, 39, 39, 35, 36, 39, 47, 27, 28, 50, 53, 36, 44, 38, 51, 43, 38, 38, 36, 40, 36, 40, 31, 39, 37, 38, 46, 28, 32, 37, 29, 26, 25,

In [22]:
assert(apache_cvs("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv") == apache_scores("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv"))
pts = apache_cvs("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv") #test patient apache II scores
answers = apache_scores("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv") #results of test patients
for (a, b) in zip(pts, answers): #this ia a for loop to look at each data element an publish if they are equal or not equal
    if a == b:
        print("The Apache II score for the test patient {} is equal to the answer is {}.".format(a, b))
    else:
        print("The Apache II score for the test patient {} is not equal to the answer is {}.".format(a, b))


The Apache II score for the test patient 35 is equal to the answer is 35.
The Apache II score for the test patient 31 is equal to the answer is 31.
The Apache II score for the test patient 47 is equal to the answer is 47.
The Apache II score for the test patient 34 is equal to the answer is 34.
The Apache II score for the test patient 44 is equal to the answer is 44.
The Apache II score for the test patient 35 is equal to the answer is 35.
The Apache II score for the test patient 31 is equal to the answer is 31.
The Apache II score for the test patient 49 is equal to the answer is 49.
The Apache II score for the test patient 40 is equal to the answer is 40.
The Apache II score for the test patient 48 is equal to the answer is 48.
The Apache II score for the test patient 42 is equal to the answer is 42.
The Apache II score for the test patient 43 is equal to the answer is 43.
The Apache II score for the test patient 32 is equal to the answer is 32.
The Apache II score for the test patie

In [23]:
import statistics
mean = round(statistics.mean(pts),2)
median = round(statistics.median(pts), 2)
std = round(statistics.pstdev(pts), 2)
mod = round(statistics.mode(pts), 2)
var = round(statistics.pvariance(pts), 2)
outcome_non = apache_outcome_data(apache, mean, "Nonoperative")
outcome_op = apache_outcome_data(apache, mean, "Operative")
print("The mean {}, median {}, standard deviation {}, mod {}, and variance {}.".format(mean, median, std, mod, var)) #this lists some statistics on the patient population
print("Using the mean and non-operative, the approximated in-hospital mortality rates would be {}.".format(outcome_non)) #uses the mean and associating with non-operative and shows what mortalities would be.
print("Using the mean and operative, the approximated in-hospital mortality rates would be {}.".format(outcome_op)) #uses the mean and associating with operative and shows what mortalities would be.

The mean 38.17, median 38.0, standard deviation 7.26, mod 40, and variance 52.67.
Using the mean and non-operative, the approximated in-hospital mortality rates would be 88%.
Using the mean and operative, the approximated in-hospital mortality rates would be 88%.
